# 🎵 Pipeline Profissional de Restauração de Áudio

Este notebook implementa um pipeline completo de restauração e masterização de áudio que inclui:

- ✅ **Análise Espectral Detalhada**
- ✅ **Redução de Ruído Avançada**
- ✅ **Restauração de Frequências**
- ✅ **Separação de Stems (opcional)**
- ✅ **Processamento Profissional**
- ✅ **Masterização Automatizada**

---

## 📦 Instalação de Dependências

In [ ]:
%%capture
# Instalar dependências necessárias
!pip install librosa soundfile scipy matplotlib numpy
!pip install noisereduce

# ⭐ DEMUCS - Separação de stems de ALTA QUALIDADE (state-of-the-art)
# Descomente a linha abaixo para instalar (recomendado se tiver GPU)
# !pip install demucs

print("✓ Dependências instaladas!")

## 💾 Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Definir caminho para a pasta 00-restore no seu Drive
AUDIO_INPUT_DIR = '/content/drive/MyDrive/00-restore'
AUDIO_OUTPUT_DIR = '/content/drive/MyDrive/00-restore/restored_output'

print(f"✓ Drive montado")
print(f"Pasta de entrada: {AUDIO_INPUT_DIR}")
print(f"Pasta de saída: {AUDIO_OUTPUT_DIR}")

# Verificar GPU (importante para Demucs)
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"\n⚡ GPU disponível: {gpu_name}")
    print(f"   VRAM: {vram:.1f} GB")
    print("   💡 Demucs será MUITO mais rápido com GPU!")
else:
    print(f"\n⚠️ GPU não disponível - usando CPU")
    print("   Para habilitar: Runtime → Change runtime type → GPU")
    print("   ⏳ Demucs será LENTO sem GPU (mas ainda funciona)")

## 📥 Baixar Módulos do Pipeline

In [ ]:
# Clone o repositório do branch correto (com todas as correções)
!git clone -b claude/audio-restoration-pipeline-gAFxk https://github.com/guitorte/musicas.git /content/audio-pipeline-repo

# Copiar módulos para o ambiente de trabalho
import sys
import shutil
from pathlib import Path

# Adicionar ao path
sys.path.insert(0, '/content/audio-pipeline-repo/audio-restoration-pipeline')

print("✓ Módulos carregados do branch correto com todas as correções!")

## 🔧 Importar Módulos

In [ ]:
import os
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Importar módulos do pipeline
from modules.pipeline import AudioRestorationPipeline
from modules.spectral_analysis import SpectralAnalyzer
from modules.frequency_restoration import FrequencyRestorer
from modules.stem_separation import StemSeparator
from modules.audio_processing import AudioProcessor

print("✓ Todos os módulos importados com sucesso!")

## ⚙️ Configuração do Pipeline

In [ ]:
# ============================================================
# CONFIGURAÇÕES DO PIPELINE
# Escolha o perfil adequado para seu áudio
# ============================================================

# ────────────────────────────────────────────────────────────
# PERFIL 1: PADRÃO - Restauração balanceada
# ────────────────────────────────────────────────────────────
CONFIG_PADRAO = {
    'remove_clicks': True,
    'reduce_noise': True,
    'noise_reduction_strength': 0.6,
    'restore_frequencies': True,
    'freq_restoration_method': 'harmonic_synthesis',
    'enhance_bass': False,
    'psychoacoustic_enhancement': True,
    'separate_stems': False,
    'target_lufs': -14.0,
    'master_eq': {
        'bass': 0.0,
        'mid': 0.0,
        'presence': 1.0,
        'treble': 0.8
    },
    'add_presence': True
}

# ────────────────────────────────────────────────────────────
# PERFIL 2: ALTA QUALIDADE - Com Demucs (RECOMENDADO!)
# ⚠️ Requer GPU, instalação do Demucs, e mais tempo
# ────────────────────────────────────────────────────────────
CONFIG_DEMUCS = {
    'remove_clicks': True,
    'reduce_noise': True,
    'noise_reduction_strength': 0.6,
    'restore_frequencies': True,
    'freq_restoration_method': 'harmonic_synthesis',
    'enhance_bass': False,
    'psychoacoustic_enhancement': True,
    'separate_stems': True,              # ⭐ Separar stems
    'stem_separation_model': 'demucs',   # ⭐ Usar Demucs
    'process_stems_individually': True,  # ⭐ Processar cada stem
    'target_lufs': -14.0,
    'master_eq': {
        'bass': 0.0,
        'mid': 0.0,
        'presence': 2.0,  # Boost em presença
        'treble': 2.5     # Boost em agudos
    },
    'add_presence': True
}

# ────────────────────────────────────────────────────────────
# PERFIL 3: RESTAURAÇÃO AGRESSIVA - Para áudio muito degradado
# ────────────────────────────────────────────────────────────
CONFIG_AGRESSIVO = {
    'remove_clicks': True,
    'reduce_noise': True,
    'noise_reduction_strength': 0.85,  # Redução forte
    'restore_frequencies': True,
    'freq_restoration_method': 'spectral_extension',
    'enhance_bass': True,
    'bass_enhancement_amount': 1.5,
    'psychoacoustic_enhancement': True,
    'separate_stems': False,
    'target_lufs': -14.0,
    'master_eq': {
        'bass': 0.5,
        'mid': -1.0,      # Reduzir médios
        'presence': 3.0,  # Boost agressivo
        'treble': 3.5     # Boost muito agressivo
    },
    'add_presence': True
}

# ────────────────────────────────────────────────────────────
# PERFIL 4: STEMS BÁSICO - Separação rápida sem Demucs
# ────────────────────────────────────────────────────────────
CONFIG_STEMS_BASICO = {
    'remove_clicks': True,
    'reduce_noise': True,
    'noise_reduction_strength': 0.6,
    'restore_frequencies': True,
    'freq_restoration_method': 'harmonic_synthesis',
    'enhance_bass': False,
    'psychoacoustic_enhancement': True,
    'separate_stems': True,
    'stem_separation_model': 'basic',    # Método básico (rápido)
    'process_stems_individually': True,
    'target_lufs': -14.0,
    'master_eq': {
        'bass': 0.0,
        'mid': 0.0,
        'presence': 2.0,
        'treble': 2.5
    },
    'add_presence': True
}

# ════════════════════════════════════════════════════════════
# SELECIONE O PERFIL AQUI:
# ════════════════════════════════════════════════════════════
CONFIG = CONFIG_DEMUCS  # ⭐ MUDE AQUI para usar outro perfil

print("✓ Configuração selecionada")
print("\n" + "═"*60)
print("PERFIL ATIVO:")
print("═"*60)
import json
print(json.dumps(CONFIG, indent=2))
print("═"*60)

# Avisos
if CONFIG.get('separate_stems') and CONFIG.get('stem_separation_model') == 'demucs':
    print("\n⚠️ ATENÇÃO: Demucs selecionado")
    print("   • Certifique-se de ter instalado: !pip install demucs")
    print("   • GPU recomendada (muito mais rápido)")
    print("   • Tempo estimado: 5-20 min por arquivo")
    
if CONFIG.get('noise_reduction_strength', 0) > 0.8:
    print("\n⚠️ Redução de ruído agressiva ativa")
    print("   • Pode criar artefatos em alguns casos")

## 📂 Listar Arquivos de Áudio

In [ ]:
# Buscar todos os arquivos MP3 e WAV
audio_files = []
audio_files.extend(glob.glob(os.path.join(AUDIO_INPUT_DIR, '*.mp3')))
audio_files.extend(glob.glob(os.path.join(AUDIO_INPUT_DIR, '*.wav')))
audio_files.extend(glob.glob(os.path.join(AUDIO_INPUT_DIR, '*.MP3')))
audio_files.extend(glob.glob(os.path.join(AUDIO_INPUT_DIR, '*.WAV')))

audio_files = sorted(set(audio_files))  # Remover duplicatas e ordenar

print(f"✓ Encontrados {len(audio_files)} arquivos de áudio:\n")
for i, file in enumerate(audio_files, 1):
    file_size_mb = os.path.getsize(file) / (1024 * 1024)
    print(f"{i:2d}. {Path(file).name} ({file_size_mb:.2f} MB)")

if not audio_files:
    print(f"⚠️ AVISO: Nenhum arquivo encontrado em {AUDIO_INPUT_DIR}")
    print("Verifique se o caminho está correto e se há arquivos MP3 ou WAV na pasta.")

## 🚀 Inicializar Pipeline

In [ ]:
# Criar diretório de saída
os.makedirs(AUDIO_OUTPUT_DIR, exist_ok=True)

# Inicializar pipeline
pipeline = AudioRestorationPipeline(
    sr=44100,
    output_base_dir=AUDIO_OUTPUT_DIR,
    log_dir=os.path.join(AUDIO_OUTPUT_DIR, 'logs')
)

print("✓ Pipeline inicializado")

## 🎯 Processar Arquivo Individual (Teste)

In [ ]:
# ============================================================
# PROCESSAR ARQUIVO INDIVIDUAL (TESTE)
# ============================================================

if audio_files:
    test_file = audio_files[0]
    print(f"Processando arquivo de teste: {Path(test_file).name}")
    print("="*70)
    
    # Avisos de tempo
    if CONFIG.get('separate_stems') and CONFIG.get('stem_separation_model') == 'demucs':
        print("⏳ DEMUCS ATIVO - Isso pode levar 5-20 minutos!")
        print("   • Separação de stems é computacionalmente intensiva")
        print("   • Com GPU: ~5-10 min")
        print("   • Sem GPU: ~15-30 min")
        print("="*70)
    
    result = pipeline.process_audio(
        test_file,
        config=CONFIG
    )
    
    print("\n" + "="*70)
    print("✓ TESTE COMPLETO!")
    print("="*70)
    print(f"\n📁 Pasta de saída: {result['output_dir']}")
    print(f"\n🎵 Arquivo final masterizado:")
    print(f"   {result['stages']['mastering']['output']}")
    
    # Mostrar stems se foram separados
    if 'stem_separation' in result['stages']:
        print(f"\n🎸 Stems separados ({len(result['stages']['stem_separation'])} arquivos):")
        for stem_name, stem_path in result['stages']['stem_separation'].items():
            print(f"   • {stem_name}: {Path(stem_path).name}")
    
    # Mostrar arquivos intermediários
    print(f"\n📊 Arquivos intermediários:")
    if 'cleanup' in result['stages']:
        print(f"   • Limpo: {Path(result['stages']['cleanup']['output']).name}")
    if 'frequency_restoration' in result['stages']:
        print(f"   • Freq. restaurado: {Path(result['stages']['frequency_restoration']['output']).name}")
    
    print("\n💡 Dica: Role para baixo para ver a análise visual!")
else:
    print("⚠️ Nenhum arquivo para processar")

## 📊 Visualizar Análise do Arquivo Processado

In [ ]:
# ============================================================
# VISUALIZAR ANÁLISE E OUVIR RESULTADOS
# ============================================================

if audio_files:
    from IPython.display import Image, Audio, display
    
    # ────────────────────────────────────────────────────────
    # 1. VISUALIZAÇÃO ESPECTRAL
    # ────────────────────────────────────────────────────────
    viz_path = os.path.join(result['output_dir'], 'analysis_visualization.png')
    if os.path.exists(viz_path):
        print("📊 ANÁLISE ESPECTRAL:")
        print("="*60)
        display(Image(filename=viz_path))
    
    # ────────────────────────────────────────────────────────
    # 2. RECOMENDAÇÕES DA ANÁLISE
    # ────────────────────────────────────────────────────────
    analysis_path = os.path.join(result['output_dir'], 'analysis.json')
    if os.path.exists(analysis_path):
        import json
        with open(analysis_path, 'r') as f:
            analysis = json.load(f)
        
        print("\n💡 RECOMENDAÇÕES DA ANÁLISE:")
        print("="*60)
        if analysis.get('recommendations'):
            for rec in analysis['recommendations']:
                severity_emoji = {
                    'high': '🔴',
                    'medium': '🟡',
                    'low': '🟢'
                }
                emoji = severity_emoji.get(rec['severity'], '⚪')
                print(f"{emoji} [{rec['severity'].upper()}] {rec['message']}")
        else:
            print("✓ Nenhum problema crítico detectado")
        
        # Mostrar métricas importantes
        print(f"\n📈 MÉTRICAS:")
        print("="*60)
        print(f"Duração: {analysis['duration']:.2f}s")
        print(f"Sample Rate: {analysis['sample_rate']} Hz")
        print(f"SNR (Signal-to-Noise): {analysis['noise_profile']['snr_db']:.1f} dB")
        print(f"LUFS estimado: {analysis['dynamic_range']['lufs_estimate']:.1f}")
        print(f"Peak: {analysis['dynamic_range']['peak_amplitude']:.3f}")
        if analysis['clipping_detection']['has_clipping']:
            print(f"⚠️ Clipping: {analysis['clipping_detection']['clip_percentage']:.2f}%")
        if analysis['frequency_analysis']['high_freq_loss']:
            print(f"⚠️ Corte de frequências em: {analysis['frequency_analysis']['high_freq_cutoff']:.0f} Hz")
    
    # ────────────────────────────────────────────────────────
    # 3. COMPARAR ORIGINAL vs PROCESSADO
    # ────────────────────────────────────────────────────────
    print("\n\n🎧 COMPARAÇÃO: ORIGINAL vs PROCESSADO")
    print("="*60)
    
    print("\n🔊 ORIGINAL:")
    display(Audio(test_file))
    
    print("\n🎵 PROCESSADO (MASTERIZADO):")
    mastered_file = result['stages']['mastering']['output']
    display(Audio(mastered_file))
    
    # ────────────────────────────────────────────────────────
    # 4. OUVIR STEMS INDIVIDUAIS (se separados)
    # ────────────────────────────────────────────────────────
    if 'stem_separation' in result['stages']:
        print("\n\n🎸 STEMS SEPARADOS")
        print("="*60)
        print("Ouça cada componente do áudio separadamente:\n")
        
        stems = result['stages']['stem_separation']
        
        if 'vocals' in stems:
            print("🎤 VOCAIS:")
            display(Audio(stems['vocals']))
        
        if 'drums' in stems:
            print("\n🥁 BATERIA:")
            display(Audio(stems['drums']))
        
        if 'bass' in stems:
            print("\n🎸 BAIXO:")
            display(Audio(stems['bass']))
        
        if 'other' in stems:
            print("\n🎹 OUTROS INSTRUMENTOS:")
            display(Audio(stems['other']))
        
        # Se tiver stems processados
        if 'processed_stems' in result['stages']:
            print("\n\n🎛️ STEMS PROCESSADOS (com EQ e compressão)")
            print("="*60)
            processed_stems = result['stages']['processed_stems']
            
            for stem_name, stem_path in processed_stems.items():
                print(f"\n✨ {stem_name.upper()} (processado):")
                display(Audio(stem_path))

## 🔄 Processar Todos os Arquivos em Batch

**⚠️ AVISO:** Isso pode levar bastante tempo dependendo da quantidade e tamanho dos arquivos!

In [ ]:
# Descomente a linha abaixo para processar TODOS os arquivos
# BATCH_PROCESS = True
BATCH_PROCESS = False  # Manter False por segurança

if BATCH_PROCESS and audio_files:
    print(f"Iniciando processamento em batch de {len(audio_files)} arquivos...")
    print("Isso pode levar vários minutos ou horas!\n")
    
    batch_results = pipeline.batch_process(
        audio_files,
        config=CONFIG
    )
    
    # Resumo
    successful = sum(1 for r in batch_results if 'error' not in r)
    failed = len(batch_results) - successful
    
    print(f"\n{'='*70}")
    print(f"BATCH COMPLETO!")
    print(f"Sucesso: {successful}/{len(batch_results)}")
    if failed > 0:
        print(f"Falhas: {failed}")
    print(f"{'='*70}")
else:
    print("Processamento em batch desabilitado.")
    print("Para processar todos os arquivos, mude BATCH_PROCESS = True na célula acima.")

# ============================================================
# INSTALAR DEMUCS (Execute se for usar CONFIG_DEMUCS)
# ============================================================

# Descomente as linhas abaixo para instalar Demucs
# %%capture
# !pip install demucs
# print("✓ Demucs instalado!")

# Verificar se Demucs está instalado
try:
    import subprocess
    result = subprocess.run(['demucs', '--help'], capture_output=True)
    print("✓ Demucs já está instalado!")
except FileNotFoundError:
    print("⚠️ Demucs NÃO instalado")
    print("   Para instalar, descomente as linhas acima e execute")
    print("   Ou execute: !pip install demucs")

In [ ]:
# ============================================================
# PROCESSAMENTO CUSTOMIZADO (Avançado)
# Use esta seção para processar com configurações personalizadas
# ============================================================

# ────────────────────────────────────────────────────────────
# EXEMPLO 1: Para áudio com muito ruído de fundo
# ────────────────────────────────────────────────────────────
CONFIG_RUIDO_PESADO = {
    'remove_clicks': True,
    'reduce_noise': True,
    'noise_reduction_strength': 0.9,    # Muito agressivo
    'restore_frequencies': True,
    'freq_restoration_method': 'harmonic_synthesis',
    'enhance_bass': False,
    'psychoacoustic_enhancement': True,
    'separate_stems': False,
    'target_lufs': -14.0,
    'master_eq': {
        'bass': 0.0,
        'mid': 0.0,
        'presence': 1.5,
        'treble': 1.5
    },
    'add_presence': True
}

# ────────────────────────────────────────────────────────────
# EXEMPLO 2: Para gravação ao vivo com baixa qualidade
# ────────────────────────────────────────────────────────────
CONFIG_GRAVACAO_AO_VIVO = {
    'remove_clicks': True,
    'reduce_noise': True,
    'noise_reduction_strength': 0.7,
    'restore_frequencies': True,
    'freq_restoration_method': 'spectral_extension',
    'enhance_bass': True,
    'bass_enhancement_amount': 1.4,
    'psychoacoustic_enhancement': True,
    'separate_stems': True,
    'stem_separation_model': 'basic',  # Rápido
    'process_stems_individually': True,
    'target_lufs': -16.0,  # Mais silencioso para broadcast
    'master_eq': {
        'bass': 1.0,
        'mid': 0.0,
        'presence': 2.5,
        'treble': 2.0
    },
    'add_presence': True
}

# ────────────────────────────────────────────────────────────
# EXEMPLO 3: Máxima qualidade - Demucs + Restauração Completa
# ────────────────────────────────────────────────────────────
CONFIG_MAXIMA_QUALIDADE = {
    'remove_clicks': True,
    'reduce_noise': True,
    'noise_reduction_strength': 0.6,
    'restore_frequencies': True,
    'freq_restoration_method': 'harmonic_synthesis',
    'enhance_bass': False,
    'psychoacoustic_enhancement': True,
    'separate_stems': True,
    'stem_separation_model': 'demucs',  # ⭐ Melhor qualidade
    'process_stems_individually': True,  # ⭐ Processar cada stem
    'target_lufs': -14.0,
    'master_eq': {
        'bass': 0.5,
        'mid': 0.0,
        'presence': 2.0,
        'treble': 2.5
    },
    'add_presence': True
}

print("✓ Configurações customizadas prontas")
print("\n💡 Para usar, descomente o código abaixo e ajuste:")
print("""
# Exemplo de uso:
# custom_file = audio_files[2]  # Escolher arquivo (índice 0, 1, 2...)
# result_custom = pipeline.process_audio(
#     custom_file,
#     config=CONFIG_MAXIMA_QUALIDADE  # ou outro perfil
# )
# print(f"✓ Processado: {result_custom['stages']['mastering']['output']}")
""")

## 💾 Download de Resultados

Os arquivos processados estão salvos em:
- **Google Drive:** `00-restore/restored_output/`

Você pode acessá-los diretamente pelo Drive ou fazer download abaixo:

In [ ]:
from google.colab import files

# Listar arquivos finais processados
output_files = glob.glob(os.path.join(AUDIO_OUTPUT_DIR, '**/*FINAL.wav'), recursive=True)

if output_files:
    print(f"Encontrados {len(output_files)} arquivos finais:\n")
    for i, file in enumerate(output_files, 1):
        print(f"{i}. {Path(file).name}")
    
    # Descomente para fazer download do primeiro arquivo
    # files.download(output_files[0])
    
    print("\n💡 Descomente a linha 'files.download()' acima para baixar um arquivo")
else:
    print("Nenhum arquivo final encontrado ainda.")

## 🔧 Utilitários Extras

In [ ]:
# Função para comparar antes/depois
def compare_audio(original_path, processed_path):
    """Compara áudio original vs processado"""
    import IPython.display as ipd
    from pathlib import Path
    
    print("🔊 ORIGINAL:")
    print(f"   {Path(original_path).name}")
    display(ipd.Audio(original_path))
    
    print("\n🎵 PROCESSADO:")
    print(f"   {Path(processed_path).name}")
    display(ipd.Audio(processed_path))

# Exemplo de uso:
# compare_audio(audio_files[0], result['stages']['mastering']['output'])

print("✓ Utilitários carregados")
print("Use compare_audio(original, processado) para comparar arquivos")

---

## 📝 Notas

- O pipeline salva múltiplas versões do áudio durante o processamento
- Análises detalhadas são salvas em JSON e PNG
- O arquivo final masterizado tem sufixo `_FINAL.wav`
- Logs completos estão disponíveis na pasta `logs/`

## 🎯 Dicas de Uso

1. **Sempre teste com um arquivo primeiro** antes de processar em batch
2. **Ajuste as configurações** baseado nos resultados da análise
3. **Redução de ruído agressiva** (>0.8) pode criar artefatos
4. **Separação de stems** é muito mais precisa com Demucs, mas requer GPU
5. **LUFS -14** é padrão para Spotify/YouTube, **-16** para TV/Rádio

---

**Pipeline desenvolvido com tecnologias de ponta em processamento de áudio**

🎵 Bom trabalho!